In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ml
from lightgbm import LGBMClassifier


# split data
from sklearn.model_selection import (KFold, 
                                     cross_val_score, 
                                     train_test_split)
# evaluate
from sklearn.metrics import accuracy_score

# optimization
import optuna

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [2]:
class Config: 
    train_path=  '/kaggle/input/playground-series-s4e6/train.csv'
    test_path = '/kaggle/input/playground-series-s4e6/test.csv'

    train_test_split = 0.1
    # cat - categorical
    # nocat - not categorical
    input_columns_params = {
        "Marital status": 'cat', 
        'Application mode': 'nocat', 
        'Application order': 'nocat', 
        'Course': 'nocat', 
        'Daytime/evening attendance': 'cat', 
        'Previous qualification': 'nocat', 
        'Previous qualification (grade)': 'nocat', 
        'Nacionality': 'cat', 
        "Mother's qualification": 'nocat', 
        "Father's qualification": 'nocat', 
        "Mother's occupation": "cat", 
        "Father's occupation": 'cat', 
        'Admission grade': 'nocat',  
        'Educational special needs': 'cat', 
        'Debtor': 'cat', 
        "Tuition fees up to date": 'cat', 
        'Gender': 'cat', 
        'Scholarship holder': 'cat', 
        'Age at enrollment': 'nocat', 
        'International': 'cat', 
        'Curricular units 1st sem (credited)': 'cat',
        'Curricular units 1st sem (enrolled)': 'cat',
        'Curricular units 1st sem (evaluations)': 'nocat',
        'Curricular units 1st sem (approved)': 'cat',
        'Curricular units 1st sem (grade)': 'nocat',
        'Curricular units 1st sem (without evaluations)': 'nocat',

        'Curricular units 2nd sem (credited)': 'cat',
        'Curricular units 2nd sem (enrolled)': 'cat',
        'Curricular units 2nd sem (evaluations)': 'nocat',
        'Curricular units 2nd sem (approved)': 'cat',
        'Curricular units 2nd sem (grade)': 'nocat',
        'Curricular units 2nd sem (without evaluations)': 'nocat',
        'Unemployment rate': 'nocat', 
        'Inflation rate': 'nocat', 
        'GDP': 'nocat', 
    }

    output_columns_params = {"Target": 'cat'}
    
cfg= Config() 

In [3]:
df = pd.read_csv(cfg.train_path)
test_df = pd.read_csv(cfg.test_path)
df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [4]:

def add_economic_interaction_features(df):    
    df['Economic_interaction_add'] = df['Unemployment rate'] * df['Inflation rate'] * df['GDP']
    df['Economic_interaction_add'] = df['Unemployment rate'] + df['Inflation rate'] + df['GDP']
    df['Economic_interaction_sub'] = df['Unemployment rate'] - df['Inflation rate'] - df['GDP']  
    df['Economic_interaction_div1'] = df['Unemployment rate'] / (df['Inflation rate'] + 1e-6)
    #df['Economic_interaction_div2'] = df['Inflation rate'] / (df['GDP'] + 1e-6)
    #df['Economic_interaction_div3'] = df['GDP'] / (df['Unemployment rate'] + 1e-6)   
    df['Economic_interaction_comb1'] = (df['Unemployment rate'] + df['Inflation rate']) * df['GDP']
    df['Economic_interaction_comb2'] = (df['Unemployment rate'] * df['Inflation rate']) + df['GDP']
    df['Economic_interaction_exp1'] = df['Unemployment rate'] * df['Inflation rate']
    df['Economic_interaction_exp2'] = df['GDP'] * df['Unemployment rate']
    df['Economic_interaction_exp3'] = df['GDP'] * df['Inflation rate']
    #df['Economic_interaction_log1'] = np.log1p(df['Unemployment rate']) + np.log1p(df['Inflation rate']) + np.log1p(df['GDP'])
    #df['Economic_interaction'] = (df['Unemployment rate'] * df['Inflation rate'] * df['GDP'])
    #df['Economic_interaction_sqrt1'] = np.sqrt(df['Unemployment rate']) + np.sqrt(df['Inflation rate']) + np.sqrt(df['GDP'])
   # df['Economic_interaction_sqrt2'] = np.sqrt(df['Unemployment rate'] * df['Inflation rate'] * df['GDP'])
    #df['Economic_interaction_poly1'] = df['Unemployment rate']**2 + df['Inflation rate']**2 + df['GDP']**2
    #df['Economic_interaction_poly2'] = (df['Unemployment rate'] * df['Inflation rate']) + (df['Inflation rate'] * df['GDP']) + (df['GDP'] * df['Unemployment rate'])
    return df     


def add_curriculum_interaction_features(df):
    df['Curriculum_interaction_1st_sem'] = df['Curricular units 1st sem (credited)'] * df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (evaluations)'] * df['Curricular units 1st sem (approved)']
    df['Curriculum_interaction_2nd_sem'] = df['Curricular units 2nd sem (credited)'] * df['Curricular units 2nd sem (enrolled)'] * df['Curricular units 2nd sem (evaluations)'] * df['Curricular units 2nd sem (approved)']

def add_grade_interaction_features(df):
    df['Grade_interaction_1st_sem'] = df['Admission grade'] * df['Curricular units 1st sem (grade)']
    df['Grade_interaction_2nd_sem'] = df['Admission grade'] * df['Curricular units 2nd sem (grade)']

def add_qualification_interaction_features(df):
    df['Qualification_interaction'] = df['Previous qualification'] * df['Admission grade']
    df['Course_interaction'] = df['Previous qualification'] * df['Course']
    
def add_occupation_interaction_features(df):
    df["Occupation_interaction"] = df["Mother's occupation"] * df["Father's occupation"]

def add_enrollment_evaluation_interaction_features(df):
    df['Enrollment_evaluation_interaction_1st_sem'] = df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (evaluations)']
    df['Enrollment_evaluation_interaction_2nd_sem'] = df['Curricular units 2nd sem (enrolled)'] * df['Curricular units 2nd sem (evaluations)']

def add_gender_marital_interaction_features(df):
    df['Gender_marital_interaction'] = df['Gender'] * df['Marital status']

def add_tuition_scholarship_interaction_features(df):
    df['Tuition_scholarship_interaction'] = df['Tuition fees up to date'] * df['Scholarship holder']    

In [5]:

def feature_engineer_train(df_train):    
    #df_train = handle_skewed_columns(df_train)   
    
    add_curriculum_interaction_features(df_train)
    add_grade_interaction_features(df_train)
#     add_age_interaction_features(df_train)
    add_economic_interaction_features(df_train)
    add_qualification_interaction_features(df_train)
    add_occupation_interaction_features(df_train)
    add_enrollment_evaluation_interaction_features(df_train)
    add_gender_marital_interaction_features(df_train)
    add_tuition_scholarship_interaction_features(df_train)
    return df_train

def prepare_X_1(df): 
    X = df[[k for k, v in cfg.input_columns_params.items()]]
    X = feature_engineer_train(X)
    # Convert real numbers for catboost 
    for i in [k for k, v in cfg.input_columns_params.items() if v == 'cat']: 
        X[i] = X[i].apply(lambda i: str(i))
        
    return X 

In [6]:
X = prepare_X_1(df)
y = df[[k for k, v in cfg.output_columns_params.items()]]

test_X = prepare_X_1(test_df)

# LGBoost & Optuna

In [7]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm 

tqdm.pandas() 

class prepare_X_2(): 
    def __init__(self): 
        self.encoders = dict()
        
        
    def one_hot_encode_column(self, df, col_name):
        """
        One-hot encodes the specified column in the given DataFrame.

        Parameters:
        df (pd.DataFrame): The input DataFrame.
        col_name (str): The name of the column to one-hot encode.

        Returns:
        pd.DataFrame: The DataFrame with the specified column one-hot encoded.
        """
        df = df.copy()
        # Initialize new the OneHotEncoder
        encoder = OneHotEncoder(sparse_output=False, drop='if_binary', handle_unknown ='ignore')  # drop='first' to avoid multicollinearity

        # Fit and transform the specified column
        encoded_col = encoder.fit(df[[col_name]])
        self.encoders[col_name] = encoder
        
        return f"{col_name} encoder succesfully fitted"
    
    def fit(self, X): 
        for i in [k for k, v in cfg.input_columns_params.items() if v == 'cat']: 
            self.one_hot_encode_column(df = X, col_name = i)
        return X 
    
    def __call__(self, X): 
        df = X.copy()
        for col_name in [k for k, v in cfg.input_columns_params.items() if v == 'cat']:
            
            encoder = self.encoders[col_name]
            encoded_col = encoder.transform(df[[col_name]])
            # Create a DataFrame with the encoded columns
            encoded_df = pd.DataFrame(encoded_col, columns=encoder.get_feature_names_out([col_name]), index=df.index)

            # Drop the original column and concatenate the encoded columns
            df = df.drop(columns=[col_name])
            df = pd.concat([df, encoded_df], axis=1)
            
        return df 
    
    def __str__(self): 
        return ", ".join([k for k, v in self.encoders.items()])

# train Encoders 
prep2 = prepare_X_2() 
prep2.fit(pd.concat([X, test_X]))

X = prep2(X)

y_uniques = y['Target'].unique()    
y['Target'] = y['Target'].progress_apply(lambda x: [i for i, v in enumerate(y_uniques) if x == v][0])

100%|██████████| 76518/76518 [00:00<00:00, 205609.09it/s]


In [8]:
def objective(trial):
    """
    Past best params 
    {
                 'objective': 'multiclass', # multiclass target: 'Graduated', 'Dropout', or 'Enrolled'
                 'data_sample_strategy': 'goss', # Gradient-based One-Sided Sampling
                 'tree_learner': 'feature', # split nodes based on the best feature
                 'n_estimators': 1743, # number of boosting iterations
                 'learning_rate': 0.02636616162598401, # step size for updatig model weights
                 'feature_fraction': 0.298183729482288, # about 30% of features considered at each split
                 'lambda_l1': 8.242410039948067e-07, # L1 regulation penalization - adding magnitude of weights to the loss
                 'lambda_l2': 0.4063299210212167, # L2 regulation penalization = adding the square of weights to the loss
                 'num_leaves': 699, # Maximum number of leaves (terminal nodes) to use
                 'max_depth': 8, # Maximum tree depth (levels) allowed
                 'colsample_bytree': 0.7975468653525116, # proportion of samples to randomly choose at each iteration
                 'min_child_samples': 102, # Minimum number of samples needed per leaf
                 'min_sum_hessian_in_leaf': 5.440582524630883, # Minimum sum of squared gradients allowed in a leaf node
                 'min_gain_to_split': 0.7247318987185962, # Minumum gain (model score improvement) to make further leaf partitions
                 'max_bin': 156, # Maximum numer of bins used for discretitizing features before tree splits
                 'top_rate': 0.6132659772851583, # Top proportion of features to choose (~61%)
                 'verbose': -1, # Turn off warnings and model logs for a cleaner look
                 'random_state': 1,  # Random state value for repeatablity,
                'early_stopping_rounds': 300, 
    }   
    
    - need to find better
    """
    
    # Suggest hyperparameters
    param = {
        'objective': 'multiclass',
        'data_sample_strategy': 'goss',
        'tree_learner': 'feature',
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 1.0),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 3.0),
        'num_leaves': trial.suggest_int('num_leaves', 599, 1024),
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 4.0, 10.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.6, 1.0),
        'max_bin': trial.suggest_int('max_bin', 150, 255),
        'top_rate': trial.suggest_float('top_rate', 0.5, 0.9),
        'verbose': -1,
        'random_state': 1,
        'early_stopping_rounds': 300
    }
    
    # K-Fold Cross-Validation
    kf = KFold(n_splits=4, shuffle=True)

    acc = list() 
    for train_size, test_size in kf.split(X, y): 
        X_train, y_train  = X.iloc[train_size], y.iloc[train_size]
        X_test, y_test = X.iloc[test_size], y.iloc[test_size]

        clf = LGBMClassifier(**param)

        # Train the model
        clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])

        # Make predictions
        y_pred = clf.predict(X_test)

        # Evaluate the model
        acc.append(accuracy_score(y_test, y_pred))
        
    accuracy = sum(acc) / len(acc)
    print("Model accuracy:", accuracy)

    return accuracy

# Create study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print best parameters
print("Best hyperparameters:", study.best_params)

[I 2024-06-26 22:13:55,458] A new study created in memory with name: no-name-0f74b218-a069-444b-8cc9-065df070d230
[I 2024-06-26 22:16:04,275] Trial 0 finished with value: 0.8302359770077847 and parameters: {'n_estimators': 1983, 'learning_rate': 0.06841194422189252, 'feature_fraction': 0.294567215984854, 'lambda_l1': 9.404722733540604, 'lambda_l2': 0.39147652766979935, 'num_leaves': 693, 'max_depth': 7, 'colsample_bytree': 0.9300541459289974, 'min_child_samples': 75, 'min_sum_hessian_in_leaf': 7.869769381738899, 'min_gain_to_split': 0.8954280328950703, 'max_bin': 171, 'top_rate': 0.8792450719592502}. Best is trial 0 with value: 0.8302359770077847.


Model accuracy: 0.8302359770077847


[I 2024-06-26 22:19:24,482] Trial 1 finished with value: 0.8314121776770953 and parameters: {'n_estimators': 1152, 'learning_rate': 0.07291289866405354, 'feature_fraction': 0.23026797149078115, 'lambda_l1': 9.751173382601966, 'lambda_l2': 1.5996227479926013, 'num_leaves': 833, 'max_depth': 9, 'colsample_bytree': 0.7511728039843475, 'min_child_samples': 54, 'min_sum_hessian_in_leaf': 9.833001164222404, 'min_gain_to_split': 0.7081363233863014, 'max_bin': 217, 'top_rate': 0.5144648038188016}. Best is trial 1 with value: 0.8314121776770953.


Model accuracy: 0.8314121776770953


[I 2024-06-26 22:22:53,364] Trial 2 finished with value: 0.8308110563443616 and parameters: {'n_estimators': 1313, 'learning_rate': 0.07735076333375954, 'feature_fraction': 0.8411830508858471, 'lambda_l1': 8.609254642676742, 'lambda_l2': 0.7788436987087267, 'num_leaves': 641, 'max_depth': 10, 'colsample_bytree': 0.6067731951924904, 'min_child_samples': 84, 'min_sum_hessian_in_leaf': 5.452427540279352, 'min_gain_to_split': 0.6012977656840507, 'max_bin': 223, 'top_rate': 0.8240614493739813}. Best is trial 1 with value: 0.8314121776770953.


...

Model accuracy: 0.8309809185589123


[I 2024-06-27 02:16:14,377] Trial 48 finished with value: 0.8310593635907001 and parameters: {'n_estimators': 1401, 'learning_rate': 0.06017922008824954, 'feature_fraction': 0.30662333395319646, 'lambda_l1': 2.6178712144014016, 'lambda_l2': 2.1607785935217323, 'num_leaves': 605, 'max_depth': 9, 'colsample_bytree': 0.6934895348382238, 'min_child_samples': 17, 'min_sum_hessian_in_leaf': 4.412812810789573, 'min_gain_to_split': 0.8583814442698061, 'max_bin': 193, 'top_rate': 0.7544821357024928}. Best is trial 10 with value: 0.8323008287720615.


Model accuracy: 0.8310593635907001


[I 2024-06-27 02:21:15,824] Trial 49 finished with value: 0.8315297960360856 and parameters: {'n_estimators': 1592, 'learning_rate': 0.0544485712171021, 'feature_fraction': 0.34282393231529995, 'lambda_l1': 0.275904688705761, 'lambda_l2': 2.7470858383952805, 'num_leaves': 629, 'max_depth': 8, 'colsample_bytree': 0.5985965514781328, 'min_child_samples': 30, 'min_sum_hessian_in_leaf': 4.9037938658417755, 'min_gain_to_split': 0.792464650238285, 'max_bin': 184, 'top_rate': 0.6867868434734641}. Best is trial 10 with value: 0.8323008287720615.


Model accuracy: 0.8315297960360856
Best hyperparameters: {'n_estimators': 1989, 'learning_rate': 0.0356738607013657, 'feature_fraction': 0.5770313576324284, 'lambda_l1': 0.3486696099396136, 'lambda_l2': 2.4285720768594317, 'num_leaves': 997, 'max_depth': 6, 'colsample_bytree': 0.6290151656741123, 'min_child_samples': 33, 'min_sum_hessian_in_leaf': 4.9588242047603215, 'min_gain_to_split': 0.9988138918047753, 'max_bin': 195, 'top_rate': 0.5833584713352055}
